In [1]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import random
import os
import cv2
from tqdm import tqdm
import image_slicer

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential, load_model
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
gpus = tf.config.experimental.list_physical_devices('GPU')

### For visualizing the outputs ###
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

print('Libraries Loaded')

Libraries Loaded


## Utilities 

In [17]:
def save_imgs(src_path,dest_path,className,coco,width,height):
    '''
    Developed by: Muhammad Saad (c)
    
    The function basically saves all the specific images to a folder. It filters out the images on the basis of the 
    class provided.
    
    Parameters:
        src_path (str): The path where the images are present. The source folder is input.
        dest_path (str): The path where you want to save the preprocessed images.
        className (str): The supercategory that will be applied to filter out the images.
        coco (obj): The module to conduct all the actions for the loading the images.
        img_input_size (tuple): The desired image size. It has to be width x height only
    
    Returns:
        unique_images (dict): The dictionary of all the images that are unique of the specified class
        catIds (list) : The list of the ids that are extracted from the class specified 
        
        The images are also saved into the specified folder
    '''
    path_old = os.getcwd()
    os.chdir(src_path)
    classes = [className]

    images = []
    if classes!=None:
        # iterate for each individual class in the list
        for className in tqdm(classes):
            # get all images containing given class
            catIds = coco.getCatIds(catNms=className)
            imgIds = coco.getImgIds(catIds=catIds)
            images += coco.loadImgs(imgIds)
    else:
        imgIds = coco.getImgIds()
        images = coco.loadImgs(imgIds)

    # Now, filter out the repeated images    
    unique_images = []
    for i in tqdm(range(len(images)),desc='Image Segregation'):
        if images[i] not in unique_images:
            unique_images.append(images[i])
            

    dataset_size = len(unique_images)

    print("Number of images containing the filter classes:", dataset_size)
    
    ## Saving the images to the destination folder
    
    for i in tqdm(range(len(unique_images)),desc='Saving Images'):
        img=cv2.resize(cv2.imread(unique_images[i]['file_name']),(height,width))
        cv2.imwrite(os.path.join(dest_path,unique_images[i]['file_name']),img)
    
    os.chdir(path_old)
    
    return unique_images,catIds

def save_ann(dest_path,unique_images,coco,cat_ids,width,height):
    '''
    Developed by: Muhammad Saad (c)
    
    The module saves the annotations to any specified folder. 
    
    Parameters:
        dest_path (str): The path where you want to save the annotations
        unique_images (dict): The dictionary of all the unique images of a specific class. 
        coco (obj) : The object for the coco which has all the variables
        cat_ids (list): The list of ids that are being extracted from the class
    
    Return:
        The images are saved into the specified folder
    '''
    path_old = os.getcwd()
    os.chdir(dest_path)
    print(f'The total images are {len(unique_images)}')
    for i in tqdm(range(len(unique_images)),desc='Annotation Saving'):
        img=unique_images[i]['id'] # Extracting the id of the image
        filename =unique_images[i]['file_name']
        ann_id=coco.getAnnIds(img, catIds=cat_ids, iscrowd=None)
        anns = coco.loadAnns(ann_id)
        mask = np.zeros((height,width))
        if len(anns)>1:
            continue
        else:
            new_mask = cv2.resize(coco.annToMask(anns[0]), (width,height))
            new_mask[new_mask >= 0.5] = 1
            new_mask[new_mask < 0.5] = 0
            mask = np.maximum(new_mask, mask)
            cv2.imwrite(filename,cv2.convertScaleAbs(mask, alpha=(255.0)))
    os.chdir(path_old)
    return

def cleanup(target_path,reference_path):
    path_old = os.getcwd()
    count=0
    for i in os.listdir(target_path):
        if i not in os.listdir(reference_path):
            os.remove(os.path.join(target_path,i))
            count+=1
    print(f'{count} files removed')
    return

def slicer(path,slices):
    path_old = os.getcwd()
    os.chdir(path)
    for i in sorted(os.listdir()):
        temp = image_slicer.slice(i,slices)
        os.remove(i)
    
    os.chdir(path_old)
    
    return

## Main

In [3]:
## Folders
root = r'C:\Users\saad\Desktop\Image-Background-Removal\COCO Dataset\COCOdataset2017'
annotations = r'annotations'
images = r'images'
pre_imgs = r'Preprocesed_images'

## Training
train_img = 'train'
train_ann = 'instances_train.json'
train_process = 'train_images'

## Validation
val_img = r'val'
val_ann = r'instances_val.json'
val_img_process = 'val_images'
val_ann_process = 'val_ann'

## Initialize the COCO api for instance annotations
# coco_train=COCO(os.path.join(root,annotation,train_ann))
coco_val=COCO(os.path.join(root,annotations,val_ann))


loading annotations into memory...
Done (t=0.78s)
creating index...
index created!


In [4]:
images,ids=save_imgs(os.path.join(root,images,val_img),os.path.join(root,pre_imgs,val_img_process),'person',coco_val,480,640)

Saving Images:   0%|▏                                                                 | 6/2693 [00:00<00:49, 54.10it/s]

Number of images containing the filter classes: 2693


Saving Images: 100%|███████████████████████████████████████████████████████████████| 2693/2693 [00:55<00:00, 48.52it/s]


In [7]:
save_ann(os.path.join(root,pre_imgs,val_ann_process),images,coco_val,ids,640,480)

Annotation Saving:   3%|█▋                                                          | 74/2693 [00:00<00:07, 361.93it/s]

The total images are 2693


Annotation Saving: 100%|██████████████████████████████████████████████████████████| 2693/2693 [00:08<00:00, 322.94it/s]


In [8]:
cleanup(os.path.join(root,pre_imgs,val_img_process),os.path.join(root,pre_imgs,val_ann_process))

1648 files removed


In [18]:
# Slicing for images
slicer(os.path.join(root,pre_imgs,val_img_process),2)
print('image slicing complete')
# Slicing for annotation
slicer(os.path.join(root,pre_imgs,val_ann_process),2)
print('annotation slicing complete')

image slicing complete
annotation slicing complete
